# Pre-Equilibrium Maximazation of Yield #

In [1]:
# make sure jupyter path is correct for loading local moudules
import sys
# path to steric_simulator module relative to notebook
sys.path.append("../../../")
import copy

In [2]:
from KineticAssembly_AD import ReactionNetwork, VectorizedRxnNet, VecSim, Optimizer, EquilibriumSolver
import networkx as nx
import torch
from torch import DoubleTensor as Tensor
import numpy as np

We'll start with the AP2 complex that we've worked with before. Pairwise $\Delta Gs$ were derived from the PDB structures via Rossetta

In [3]:
base_input = '../input_files/tetramer_diversification.pwr'
rn = ReactionNetwork(base_input, one_step=True)
rn.resolve_tree()
vec_rn = VectorizedRxnNet(rn, dev='cpu')

['default_assoc', 1.0]
['rxn_coupling', True]
True
['monomer_add_only', False]
[(0, {'struct': <networkx.classes.graph.Graph object at 0x7fa61c1d40b8>, 'copies': tensor([100.], dtype=torch.float64), 'subunits': 1}), (1, {'struct': <networkx.classes.graph.Graph object at 0x7fa6a15e3f28>, 'copies': tensor([100.], dtype=torch.float64), 'subunits': 1}), (2, {'struct': <networkx.classes.graph.Graph object at 0x7fa6a15e36a0>, 'copies': tensor([100.], dtype=torch.float64), 'subunits': 1}), (3, {'struct': <networkx.classes.graph.Graph object at 0x7fa6a15e3f60>, 'copies': tensor([100.], dtype=torch.float64), 'subunits': 1})]
New node added - Node index: 4 ; Node label: AM 
New node added - Node index: 5 ; Node label: AB 
New node added - Node index: 6 ; Node label: AS 
New node added - Node index: 7 ; Node label: BM 
New node added - Node index: 8 ; Node label: MS 
New node added - Node index: 9 ; Node label: ABM 
New node added - Node index: 10 ; Node label: AMS 
New node added - Node index: 1

In [4]:
uid_dict = {}
reaction_scheme = {}
sys.path.append("../../")
import numpy as np
from reaction_network import gtostr
for n in rn.network.nodes():
    #print(n)
    #print(rn.network.nodes()[n])
    print(n,"--",gtostr(rn.network.nodes[n]['struct']))
    for k,v in rn.network[n].items():
        uid = v['uid']
        r1 = set(gtostr(rn.network.nodes[n]['struct']))
        p = set(gtostr(rn.network.nodes[k]['struct']))
        r2 = p-r1
        reactants = (r1,r2)
        uid_dict[(n,k)] = uid
        reaction_scheme[uid] = ("".join(r1),"".join(r2),"".join(p))

print(uid_dict)

0 -- A
1 -- M
2 -- B
3 -- S
4 -- AM
5 -- AB
6 -- AS
7 -- BM
8 -- MS
9 -- ABM
10 -- AMS
11 -- BS
12 -- ABS
13 -- BMS
14 -- ABMS
{(0, 4): 0, (0, 5): 1, (0, 6): 2, (0, 9): 19, (0, 10): 20, (0, 12): 21, (0, 14): 24, (1, 4): 0, (1, 7): 3, (1, 8): 4, (1, 9): 5, (1, 10): 6, (1, 13): 22, (1, 14): 23, (2, 5): 1, (2, 7): 3, (2, 11): 7, (2, 9): 8, (2, 12): 9, (2, 13): 10, (2, 14): 11, (3, 6): 2, (3, 8): 4, (3, 11): 7, (3, 10): 12, (3, 12): 13, (3, 13): 14, (3, 14): 15, (4, 9): 8, (4, 10): 12, (4, 14): 16, (5, 9): 5, (5, 12): 13, (5, 14): 17, (6, 10): 6, (6, 12): 9, (6, 14): 18, (7, 13): 14, (7, 14): 18, (7, 9): 19, (8, 13): 10, (8, 14): 17, (8, 10): 20, (9, 14): 15, (10, 14): 11, (11, 14): 16, (11, 12): 21, (11, 13): 22, (12, 14): 23, (13, 14): 24}


In [5]:
print(rn.rxn_class)

{(1, 1): [0, 1, 2, 3, 4, 7], (2, 1): [5, 6, 8, 9, 10, 12, 13, 14, 19, 20, 21, 22], (3, 1): [11, 15, 23, 24], (2, 2): [16, 17, 18]}


In [6]:
#Do modifications here
#Changing Initial Conditions
import networkx as nx
#Changin k_on
new_kon = torch.zeros([rn._rxn_count], requires_grad=True).double()
new_kon = new_kon + Tensor([1]*np.array(1e-1))
# new_kon[rn.rxn_class[(1,1)]]=torch.Tensor([5.0837, 5.2373, 4.9745, 0.2444, 0.3391, 0.2603]).double()

# new_kon[0:3]=2

update_kon_dict = {}
for edge in rn.network.edges:
    print(rn.network.get_edge_data(edge[0],edge[1]))
    update_kon_dict[edge] = new_kon[uid_dict[edge]]

nx.set_edge_attributes(rn.network,update_kon_dict,'k_on')


# optim_rates=[0,1,2]
vec_rn = VectorizedRxnNet(rn, dev='cpu',optim_rates=[0,1,2])
print(vec_rn.kon)



{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 0}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 1}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 2}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-40.], dtype=torch.float64), 'uid': 19}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-40.], dtype=torch.float64), 'uid': 20}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-40.], dtype=torch.float64), 'uid': 21}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-60.], dtype=torch.float64), 'uid': 24}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 0}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 3}
{'k_on': 1.0, 'k_off': None, 'lcf': 1, 'rxn_score': tensor([-20.], dtype=torch.float64), 'uid': 4}
{'k_on

In [7]:
print(vec_rn.rxn_class)

{(1, 1): [0, 1, 2, 3, 4, 7], (2, 1): [5, 6, 8, 9, 10, 12, 13, 14, 19, 20, 21, 22], (3, 1): [11, 15, 23, 24], (2, 2): [16, 17, 18]}


## Using the optimizer ##

### Define an instance of the optimizer class
#### Input Arguments:

reaction_network : Input the vectorized rxn network

sim_runtime: The runtime of the kinetic simulation. Needs to be same as the time over the experimental reaction data.

optim_iterations: No. of iterations to run the optimization. Can start at low values(100) and increase depending upon memory usage.

learning_rate = The size of the gradient descent step for updating parameter values. Needs to be atleast (1e-3-1e-1)* min{parameter value}. If learning rate is too high, it can take a longer step and sometimes lead to negative value of parameters which is unphysical. Requires some trial runs to find the best value. 

device: cpu or gpu

method: Choose which pytorch based optimized to use for gradient descent - Adam or RMSprop

mom: Only for RMSprop method. Use momentum term during gradient descent. 


In [8]:
vec_rn.reset(reset_params=True)
optim = Optimizer(reaction_network=vec_rn,
                  sim_runtime=1,
                  optim_iterations=5000,
                  learning_rate=1e-2,
                  device='cpu',method="RMSprop",)


### Call the optimization method

#### Input arguments

conc_scale, conc_thresh, mod_bool, mod_factor = As defined for the VecSim class. 

max_thresh: Max. allowed values of parameters being updated. Beyond this maximum a penalty is imposed on the cost function. (Regularization)

max_yield: It is a control variable that is used to store the updated parameter values over all iterations for further analysis. The parameter values are stored only if the current yield exceed this max_yield. 

yield_species: Yield of the species being optimized(node index). Default value is -1 (picks the last element from the array which is always the final complex for a fully connected topology)

In [ ]:
optim.rn.update_reaction_net(rn)
optim.optimize(conc_scale=1,conc_thresh=1,max_thresh=10,yield_species=14)

Be careful about choosing yield_species; It defaults to the largest complex
Reaction Parameters before optimization: 
[Parameter containing:
tensor([0.1000, 0.1000, 0.1000], dtype=torch.float64, requires_grad=True)]
Optimizer State: <bound method Optimizer.state_dict of RMSprop (
Parameter Group 0
    alpha: 0.99
    centered: False
    eps: 1e-08
    lr: 0.01
    momentum: 0
    weight_decay: 0
)>
Using CPU
SLow rates:  tensor([[0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Yield on sim. iteration 0 was 41.6%.
current params: tensor([0.1000, 0.1000, 0.1000], dtype=torch.float64)
Using CPU
SLow rates:  tensor([[2.0000e-01, 2.0000e-01, 2.0071e-07, 1.0000e-01, 1.0000e-01, 2.0000e-01,
         2.0000e-01, 1.0000e-01, 2.0000e-01, 2.0000e-01, 1.0000e-01, 2.0000e-

Yield on sim. iteration 16 was 65.0%.
current params: tensor([0.3930, 0.5647, 0.1052], dtype=torch.float64)
tensor(0.6500, dtype=torch.float64, grad_fn=<DivBackward0>)
Using CPU
SLow rates:  tensor([[0.4006, 0.5743, 0.1057, 0.1000, 0.1000, 0.4006, 0.4006, 0.1000, 0.5743,
         0.5743, 0.1000, 0.5743, 0.1057, 0.1057, 0.1000, 0.1057, 0.5743, 0.4006,
         0.5743, 0.5743, 0.4006, 0.5743, 0.1000, 0.4006, 0.5743]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Yield on sim. iteration 17 was 65.1%.
current params: tensor([0.4006, 0.5743, 0.1057], dtype=torch.float64)
tensor(0.6517, dtype=torch.float64, grad_fn=<DivBackward0>)
Using CPU
SLow rates:  tensor([[0.4080, 0.5834, 0.1062, 0.1000, 0.1000, 0.4080, 0.4080, 0.1000, 0.5834,
         0.5834, 0.1000, 0.5834, 0.1062, 0.1062, 0.1000, 0.1062, 0.5834, 0.4080,
         0.5834, 0.5834, 0.4080, 0.5834, 0.1000, 0.4080, 0.5834]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Yield on sim. iteration 18 was 66.2%.
curren

Yield on sim. iteration 34 was 68.8%.
current params: tensor([0.4988, 0.6903, 0.1153], dtype=torch.float64)
tensor(0.6881, dtype=torch.float64, grad_fn=<DivBackward0>)
Using CPU
SLow rates:  tensor([[0.5034, 0.6954, 0.1159, 0.1000, 0.1000, 0.5034, 0.5034, 0.1000, 0.6954,
         0.6954, 0.1000, 0.6954, 0.1159, 0.1159, 0.1000, 0.1159, 0.6954, 0.5034,
         0.6954, 0.6954, 0.5034, 0.6954, 0.1000, 0.5034, 0.6954]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Yield on sim. iteration 35 was 68.8%.
current params: tensor([0.5034, 0.6954, 0.1159], dtype=torch.float64)
tensor(0.6888, dtype=torch.float64, grad_fn=<DivBackward0>)
Using CPU
SLow rates:  tensor([[0.5078, 0.7003, 0.1165, 0.1000, 0.1000, 0.5078, 0.5078, 0.1000, 0.7003,
         0.7003, 0.1000, 0.7003, 0.1165, 0.1165, 0.1000, 0.1165, 0.7003, 0.5078,
         0.7003, 0.7003, 0.5078, 0.7003, 0.1000, 0.5078, 0.7003]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Yield on sim. iteration 36 was 68.9%.
curren

Yield on sim. iteration 52 was 70.7%.
current params: tensor([0.4649, 0.8614, 0.2258], dtype=torch.float64)
tensor(0.7071, dtype=torch.float64, grad_fn=<DivBackward0>)
Using CPU
SLow rates:  tensor([[0.4037, 0.7666, 0.2660, 0.1000, 0.1000, 0.4037, 0.4037, 0.1000, 0.7666,
         0.7666, 0.1000, 0.7666, 0.2660, 0.2660, 0.1000, 0.2660, 0.7666, 0.4037,
         0.7666, 0.7666, 0.4037, 0.7666, 0.1000, 0.4037, 0.7666]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Yield on sim. iteration 53 was 68.5%.
current params: tensor([0.4037, 0.7666, 0.2660], dtype=torch.float64)
tensor(0.6859, dtype=torch.float64, grad_fn=<DivBackward0>)
Using CPU
SLow rates:  tensor([[0.3865, 0.7763, 0.2568, 0.1000, 0.1000, 0.3865, 0.3865, 0.1000, 0.7763,
         0.7763, 0.1000, 0.7763, 0.2568, 0.2568, 0.1000, 0.2568, 0.7763, 0.3865,
         0.7763, 0.7763, 0.3865, 0.7763, 0.1000, 0.3865, 0.7763]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Yield on sim. iteration 54 was 68.4%.
curren

Yield on sim. iteration 70 was 64.5%.
current params: tensor([0.2194, 0.7990, 0.1770], dtype=torch.float64)
tensor(0.6450, dtype=torch.float64, grad_fn=<DivBackward0>)
Using CPU
SLow rates:  tensor([[0.2194, 0.7990, 0.1770, 0.1000, 0.1000, 0.2194, 0.2194, 0.1000, 0.7990,
         0.7990, 0.1000, 0.7990, 0.1770, 0.1770, 0.1000, 0.1770, 0.7990, 0.2194,
         0.7990, 0.7990, 0.2194, 0.7990, 0.1000, 0.2194, 0.7990]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Yield on sim. iteration 71 was 64.5%.
current params: tensor([0.2194, 0.7990, 0.1770], dtype=torch.float64)
tensor(0.6450, dtype=torch.float64, grad_fn=<DivBackward0>)
Using CPU
SLow rates:  tensor([[0.2194, 0.7990, 0.1770, 0.1000, 0.1000, 0.2194, 0.2194, 0.1000, 0.7990,
         0.7990, 0.1000, 0.7990, 0.1770, 0.1770, 0.1000, 0.1770, 0.7990, 0.2194,
         0.7990, 0.7990, 0.2194, 0.7990, 0.1000, 0.2194, 0.7990]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Yield on sim. iteration 72 was 64.5%.
curren

...

Using CPU
SLow rates:  tensor([[0.3410, 2.3020, 0.3394, 0.1000, 0.1000, 0.3410, 0.3410, 0.1000, 2.3020,
         2.3020, 0.1000, 2.3020, 0.3394, 0.3394, 0.1000, 0.3394, 2.3020, 0.3410,
         2.3020, 2.3020, 0.3410, 2.3020, 0.1000, 0.3410, 2.3020]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Yield on sim. iteration 2067 was 74.8%.
current params: tensor([0.3410, 2.3020, 0.3394], dtype=torch.float64)
tensor(0.7489, dtype=torch.float64, grad_fn=<DivBackward0>)
Using CPU
SLow rates:  tensor([[0.3195, 2.3229, 0.3199, 0.1000, 0.1000, 0.3195, 0.3195, 0.1000, 2.3229,
         2.3229, 0.1000, 2.3229, 0.3199, 0.3199, 0.1000, 0.3199, 2.3229, 0.3199,
         2.3229, 2.3229, 0.3199, 2.3229, 0.1000, 0.3195, 2.3229]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Yield on sim. iteration 2068 was 74.2%.
current params: tensor([0.3195, 2.3229, 0.3199], dtype=torch.float64)
tensor(0.7426, dtype=torch.float64, grad_fn=<DivBackward0>)
Using CPU
SLow rates:  tensor([[0.3281, 

Using CPU
SLow rates:  tensor([[0.3245, 2.3220, 0.3243, 0.1000, 0.1000, 0.3245, 0.3245, 0.1000, 2.3220,
         2.3220, 0.1000, 2.3220, 0.3243, 0.3243, 0.1000, 0.3243, 2.3220, 0.3245,
         2.3220, 2.3220, 0.3245, 2.3220, 0.1000, 0.3245, 2.3220]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Yield on sim. iteration 2085 was 74.3%.
current params: tensor([0.3245, 2.3220, 0.3243], dtype=torch.float64)
tensor(0.7435, dtype=torch.float64, grad_fn=<DivBackward0>)
Using CPU
SLow rates:  tensor([[0.3365, 2.3109, 0.3354, 0.1000, 0.1000, 0.3365, 0.3365, 0.1000, 2.3109,
         2.3109, 0.1000, 2.3109, 0.3354, 0.3354, 0.1000, 0.3354, 2.3109, 0.3365,
         2.3109, 2.3109, 0.3365, 2.3109, 0.1000, 0.3365, 2.3109]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Yield on sim. iteration 2086 was 74.7%.
current params: tensor([0.3365, 2.3109, 0.3354], dtype=torch.float64)
tensor(0.7479, dtype=torch.float64, grad_fn=<DivBackward0>)
Using CPU
SLow rates:  tensor([[0.3293, 

Using CPU
SLow rates:  tensor([[0.3260, 2.3238, 0.3267, 0.1000, 0.1000, 0.3260, 0.3260, 0.1000, 2.3238,
         2.3238, 0.1000, 2.3238, 0.3267, 0.3267, 0.1000, 0.3267, 2.3238, 0.3267,
         2.3238, 2.3238, 0.3267, 2.3238, 0.1000, 0.3260, 2.3238]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Yield on sim. iteration 2103 was 74.4%.
current params: tensor([0.3260, 2.3238, 0.3267], dtype=torch.float64)
tensor(0.7440, dtype=torch.float64, grad_fn=<DivBackward0>)
Using CPU
SLow rates:  tensor([[0.3389, 2.3109, 0.3403, 0.1000, 0.1000, 0.3389, 0.3389, 0.1000, 2.3109,
         2.3109, 0.1000, 2.3109, 0.3403, 0.3403, 0.1000, 0.3403, 2.3109, 0.3403,
         2.3109, 2.3109, 0.3403, 2.3109, 0.1000, 0.3389, 2.3109]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Yield on sim. iteration 2104 was 74.8%.
current params: tensor([0.3389, 2.3109, 0.3403], dtype=torch.float64)
tensor(0.7489, dtype=torch.float64, grad_fn=<DivBackward0>)
Using CPU
SLow rates:  tensor([[0.3224, 

Yield on sim. iteration 2121 was 74.6%.
current params: tensor([0.3288, 2.3256, 0.3290], dtype=torch.float64)
tensor(0.7463, dtype=torch.float64, grad_fn=<DivBackward0>)
Using CPU
SLow rates:  tensor([[0.3263, 2.3284, 0.3260, 0.1000, 0.1000, 0.3263, 0.3263, 0.1000, 2.3284,
         2.3284, 0.1000, 2.3284, 0.3260, 0.3260, 0.1000, 0.3260, 2.3284, 0.3263,
         2.3284, 2.3284, 0.3263, 2.3284, 0.1000, 0.3263, 2.3284]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Yield on sim. iteration 2122 was 74.4%.
current params: tensor([0.3263, 2.3284, 0.3260], dtype=torch.float64)
tensor(0.7440, dtype=torch.float64, grad_fn=<DivBackward0>)
Using CPU
SLow rates:  tensor([[0.3396, 2.3159, 0.3382, 0.1000, 0.1000, 0.3396, 0.3396, 0.1000, 2.3159,
         2.3159, 0.1000, 2.3159, 0.3382, 0.3382, 0.1000, 0.3382, 2.3159, 0.3396,
         2.3159, 2.3159, 0.3396, 2.3159, 0.1000, 0.3396, 2.3159]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Yield on sim. iteration 2123 was 74.8%.


Yield on sim. iteration 2139 was 74.4%.
current params: tensor([0.3264, 2.3320, 0.3261], dtype=torch.float64)
tensor(0.7441, dtype=torch.float64, grad_fn=<DivBackward0>)
Using CPU
SLow rates:  tensor([[0.3398, 2.3195, 0.3385, 0.1000, 0.1000, 0.3398, 0.3398, 0.1000, 2.3195,
         2.3195, 0.1000, 2.3195, 0.3385, 0.3385, 0.1000, 0.3385, 2.3195, 0.3398,
         2.3195, 2.3195, 0.3398, 2.3195, 0.1000, 0.3398, 2.3195]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Yield on sim. iteration 2140 was 74.8%.
current params: tensor([0.3398, 2.3195, 0.3385], dtype=torch.float64)
tensor(0.7489, dtype=torch.float64, grad_fn=<DivBackward0>)
Using CPU
SLow rates:  tensor([[0.3232, 2.3356, 0.3235, 0.1000, 0.1000, 0.3232, 0.3232, 0.1000, 2.3356,
         2.3356, 0.1000, 2.3356, 0.3235, 0.3235, 0.1000, 0.3235, 2.3356, 0.3235,
         2.3356, 2.3356, 0.3235, 2.3356, 0.1000, 0.3232, 2.3356]],
       dtype=torch.float64, grad_fn=<UnsqueezeBackward0>)
Yield on sim. iteration 2141 was 74.3%.


## Track the yield over optim iterations

In [ ]:
optim.plot_yield()

In [ ]:
yields= []
final_params=[]
for i in range(len(optim.final_yields)):
    yields.append(optim.final_yields[i])
#     print(optim.final_solns[i].numpy())
    final_params.append(optim.final_solns[i])

sort_indx=np.argsort(np.array(yields))
sorted_yields=np.array(yields)[sort_indx]
sorted_params = np.array(final_params)[sort_indx]

p0 = sorted_params[0]
var_params = []
for i in range(len(sorted_params)):
    var_params.append(calc_var(p0,sorted_params[i]))
    
arg_indx = np.argsort(np.array(var_params))
sorted_var = np.array(var_params)[arg_indx]

print(sorted_var[0])
print(sorted_var[-1])
print("Yield: ",sorted_yields[arg_indx[0]],"\nParams: ",sorted_params[arg_indx[0]])

print("Yield: ",sorted_yields[arg_indx[-1]],"\nParams: ",sorted_params[arg_indx[-1]])
print("Max Yield: ",sorted_yields[-1],"\nParams: ",sorted_params[-1])

In [ ]:
print(vec_rn.rx_cid)
print(vec_rn.coup_map)

In [ ]:
yields= []
final_params=[]
asymm = []
final_t50 = []
final_t85 = []
final_t95 = []
final_t99 = []
for i in range(len(optim.final_yields)):
    yields.append(optim.final_yields[i])
#     print(optim.final_solns[i].numpy())
    coupled_params=np.zeros(len(vec_rn.kon))
    if vec_rn.partial_opt:
        for j in range(len(vec_rn.kon)):
            if j in vec_rn.rx_cid.keys():
                all_rates=[]
                for rate in vec_rn.rx_cid[j]:
                    if rate in vec_rn.optim_rates:
                        all_rates.append(optim.final_solns[i][vec_rn.coup_map[rate]])
                    else:
                        if vec_rn.slow_rates is not None and rate in vec_rn.slow_rates:
                            all_rates.append(torch.mean(optim.final_solns[i][vec_rn.optim_rates])/vec_rn.slow_ratio)
                        else:
                            all_rates.append(vec_rn.kon[rate])
                coupled_params[j] = max(all_rates)
            else:
                if j in vec_rn.optim_rates:
                    coupled_params[j] = optim.final_solns[i][vec_rn.coup_map[j]]
                else:
                    if vec_rn.slow_rates is not None and j in vec_rn.slow_rates:       #Can be replaced later so that the RN figures out by iteself which are fast  interfaces and which are slow.
                        coupled_params[j] = torch.mean(optim.final_solns[i][vec_rn.optim_rates])/vec_rn.slow_ratio
                    else:
                        coupled_params[j] = vec_rn.kon[j]
    else:
        for j in range(len(vec_rn.kon)):
            if j in vec_rn.rx_cid.keys():
                #new_kon[i] = 1.0
                coupled_params[j] = max(optim.final_solns[i].numpy()[vec_rn.coup_map[rate]] for rate in vec_rn.rx_cid[j])
                # print("Max rate for reaction %s chosen as %.3f" %(i,self.coupled_kon[i]))
            else:
                coupled_params[j] = optim.final_solns[i].numpy()[vec_rn.coup_map[j]]
    final_params.append(coupled_params)
    
    if type(optim.final_t50[i])==int:
        final_t50.append(1) 
    else:
        final_t50.append(optim.final_t50[i].item()) 
    if type(optim.final_t85[i])==int:
        final_t85.append(1) 
    else:
        final_t85.append(optim.final_t85[i].item()) 
    if type(optim.final_t95[i])==int:
        final_t95.append(1)
    else:
        final_t95.append(optim.final_t95[i].item())

final_yield_arr = np.array(yields)
final_param_arr = np.array(final_params)
final_t85 = np.array(final_t85)
final_t95 = np.array(final_t95)



In [ ]:
import matplotlib.pyplot as plt

mask_r = final_yield_arr > 0.5

#Calculate the ratio
ratio = final_param_arr[:,1]/final_param_arr[:,0]

#Normalize the time scale (t = t*conc*max_rate)
conc=vec_rn.initial_copies[0].item()
scale_time = final_t95[mask_r]*conc*np.max(final_param_arr[mask_r],axis=1)
#Calculate the y_per_time
y_per_time = 0.95/scale_time

fig,ax = plt.subplots(figsize=(8,6))
ax.plot(ratio,y_per_time,linestyle='',marker='o')
ax.set_ylabel("Efficiency",fontsize=20)
ax.set_xlabel("Ratio",fontsize=20)
ax.tick_params(labelsize=25)


In [ ]:
print("%5s\t\t%10s\t\t%10s" %("Uid","Reactants","Product"))
for uid,species in sorted(reaction_scheme.items()):

    print("%5d\t\t%4s + %4s\t\t%10s" %(uid,species[0],species[1],species[2]) )